## Introduction

In this notebook, we will try to obtain the unique headers amongst all the csv files that we are going to merge. Then we will finally make a master comma seperated value file with list of headers that we obtain from this notebook

## Data Cleaning

### Import the necessary libaries 

In [14]:
import pandas as pd
import numpy as np
import glob
import csv
import re
import os

### Setting up parent directory and sub directory

In [15]:
parent_dir = "../Data/"
filelist = []
dirs = []

def makefilelist(parent_dir):
    headers = []
    csv_headers = []
    subject_dirs = [os.path.join(parent_dir, dir) for dir in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, dir))]
    filelist = []
    for dir in subject_dirs:
        csv_files = [os.path.join(dir, csv) for csv in os.listdir(dir) if os.path.isfile(os.path.join(dir, csv)) and csv.endswith('.csv')]
        for file in csv_files:
            filelist.append(file)
    
    return filelist, subject_dirs

### Read headers from CSV files

In [16]:
def readCSV(fileList, subject_dirs):
    master_csv_headers = []
    for filename in fileList:
        slash = filename.split("/")
        parts = slash[2]
        subparts = parts.split("_")
        CIK = subparts[0]
        report_type = subparts[1]
        subsubpart = subparts[2].split('-')
        report_year = subsubpart[1]    
        df = pd.read_csv(filename, engine='python', error_bad_lines = False)
        df['CIK'] = CIK
        df['Reporting Type'] = report_type
        df['Report Year'] = report_year
        df.to_csv(filename, encoding='utf-8', index=False)
        
        with open(filename, 'r') as f:
            d_reader = csv.DictReader(f)
            headers = d_reader.fieldnames
            for header in headers:
                master_csv_headers.append(header)
            
    return master_csv_headers

In [17]:
filelist, dirs = makefilelist(parent_dir)
csv_headers = readCSV(filelist, dirs)

Now we will only use the unqiue headers that we might require for our master CSV

In [18]:
def uniqueHeaders(csv_headers):
    return set(csv_headers)

def chomp(list1):
    list1 = [x.replace('\n', '') for x in list1]
    return list1

Checking our unique header values and taking a look on the kind of values we have

In [19]:
final_headers = uniqueHeaders(csv_headers)
print(len(uniqueHeaders(csv_headers)))
lol = chomp(final_headers)
print(len(lol))

2807
2807


### Checking for different instances of similar headers

Using this to test out the best conditions required to extract all the possible variants of each header in the unified CSV

In [20]:
counter = 0
for header in lol:
    if 'des' in lol:
        print(header)
        counter+=1
print ("Counter", counter)

Counter 0


We will usse the above function as a way to guage how many similar ways are there to report a specific header and then merge all the occurances together to the master csv when we are conducting the joining

### Normalizing the values present in the column

In [12]:
def normalizeValues(filelist):
    for filename in filelist:
        with open(filename, 'r') as f:
            df = pd.read_csv(filename, engine='python', dtype=str)
            headers = list(df)
            for header in headers:
                if '000' in header:
                    df[header] = df[header].astype(str) + '000'
            print (df)
            print("Filename", filename)        
            df.to_csv(filename, index=False, header=True)

In [13]:
normalizeValues(filelist)

      \nCounterparty\n                  \nReference Entity / Obligation\n  \
0         \nBarclays\n           \nEastman Chemical Co, 7.60%, 02/01/27\n   
1             Barclays         \nHost Hotels & Resorts, 4.75%, 03/01/23\n   
2             Barclays        \nMacy's Retail Holdings, 7.45%, 07/15/17\n   
3      \nBNP Paribas\n         \nHost Hotels & Resorts, 4.75%, 03/01/23\n   
4          BNP Paribas        \nInternational Paper Co, 7.50%, 08/15/21\n   
5          BNP Paribas  \nMarriott International, Inc., 3.00%, 03/01/19\n   
6          BNP Paribas             \nRyder Systems Inc, 2.55%, 06/01/19\n   
7    \nGoldman Sachs\n         \nHost Hotels & Resorts, 4.75%, 03/01/23\n   
8        Goldman Sachs                 \nNordstrom Inc, 6.95%, 03/15/28\n   
9        Goldman Sachs             \nRyder Systems Inc, 2.55%, 06/01/19\n   
10  \nMorgan Stanley\n                    \nGatx Corp., 6.00%, 02/15/18\n   
11      Morgan Stanley        \nInternational Paper Co, 7.50%, 08/15/21\n   

     Reference Entity     Date Counterparty1 Amount($000) Fixed Rate(%)  \
0   CMBX.NA.AAA 6/A32  5/11/63          DBAG     13900000           0.5   
1   CMBX.NA.AAA 6/A32  5/11/63          MSCS     13205000           0.5   
2   CMBX.NA.AAA 6/A32  5/11/63         UBSAG     13010000           0.5   
3   CMBX.NA.AAA 6/A32  5/11/63           GSI     13900000           0.5   
4   CMBX.NA.AAA 6/A32  5/11/63          DBAG     13900000           0.5   
5   CMBX.NA.AAA 6/A32  5/11/63         CSFBI     13110000           0.5   
6   CMBX.NA.AAA 6/A32  5/11/63          DBAG     12950000           0.5   
7   CMBX.NA.AAA 6/A32  5/11/63          DBAG     25900000           0.5   
8   CMBX.NA.AAA 6/A32  5/11/63          MSCS     24970000           0.5   
9   CMBX.NA.AAA 6/A32  5/11/63         CSFBI     26315000           0.5   
10  CMBX.NA.AAA 6/A32  5/11/63          MSCS     13205000           0.5   

   Appreciation($000)         CIK Reporting Type Report Year  
0               88000  0000105563   

  Clearing House      Reference Entity Buy/SellProtection (7)  \
0      Citigroup  MarkitCDXNAHY21Index                   Sell   

  CurrentCreditSpread (8) NotionalAmount Fixed Rate(Annualized)  \
0                     3.5      1,800,000                    5.0   

  TerminationDate    Value UnrealizedAppreciation(Depreciation)         CIK  \
0        12/20/18  128,585                                3,972  0001390204   

  Reporting Type Report Year  
0          N-CSR          14  
Filename ../Data-CSR/0001390204_N-CSR_0001193125-14-135290/table2.csv
                        Reference Entity Counterparty Protection Amount(000s)  \
0  Morgan Stanley Capital Services, Inc.          Buy        240      0.63000   
1  Morgan Stanley Capital Services, Inc.          Buy        240       0.7000   
2                      Barclays Bank plc         Sell        460      4.15000   
3            Credit Suisse International         Sell        230      4.15000   
4  Morgan Stanley Capital Services, In

                               Underlying Instrument    Counterparty  \
0  CDX North America Investment Grade Index Swap ...  JPMorgan Chase   
1  iTraxx Crossover Swap Agreement with JPMorgan ...  JPMorgan Chase   

  ExpirationDate ImpliedCreditSpread\natDecember 31,2015 NotionalAmount($)(e)  \
0       12/20/20                                    0.88            4,802,000   
1       12/20/20                                    1.36            2,074,269   

  FixedRate(%) Value($) UpfrontPremiumsPaid/(Received)($)  \
0          1.0   25,310                            39,107   
1          5.0  187,897                           126,469   

  UnrealizedAppreciation/(Depreciation)($)         CIK Reporting Type  \
0                                  (13,797  0001301708          N-CSR   
1                                   61,428  0001301708          N-CSR   

  Report Year  
0          16  
1          16  
Filename ../Data-CSR/0001301708_N-CSR_0001193125-16-498582/table4.csv
                 

            \nCounterparty   \nReferenceEntity \nBuy/Sell  \
0  \nLehman Brothers, Inc.  Republic of Brazil      \nBuy   
1  \nLehman Brothers, Inc.   \nCDX.EM.8 Index*      \nBuy   

  \nNotionalAmount(000s omitted) \nPay/ReceiveAnnualFixedRate  \
0                     \n1,057000                         1.45   
1                     \n2,000000                         1.75   

  \nTerminationDate \nNetUnrealizedAppreciation(Depreciation)         CIK  \
0         \n7/20/12                                   \n4,062  0001394395   
1        \n12/20/12                                  \n(3,610  0001394395   

  Reporting Type Report Year  
0          N-CSR          07  
1          N-CSR          07  
Filename ../Data-CSR/0001394395_N-CSR_0001104659-07-091427/table5.csv
            Counterparty              Reference Obligation Protection2  \
0     Barclays Group PLC    Lehman Brothers Holdings, Inc.        Sold   
1    Goldman Sachs & Co.                   Nordstrom Inc.    Purchased   
2  

...

  ReferenceEntity/Obligation TerminationDate FixedRate  \
0            French Republic        03/20/13      0.25   
1     Kingdom of Netherlands        03/20/13      0.25   
2         Republic ofAustria        03/20/13       1.0   

  NotionalAmount(000)#(3) ImpliedCreditSpread atApril 30,2012(4) FairValue  \
0                  200000                                  0.922    (1,138   
1                  200000                                  0.561      (497   
2                  200000                                  0.732       711   

  UpfrontPremiumsPaid/(Received) UnrealizedAppreciation  \
0                         (2,137                  $ 999   
1                           (840                    343   
2                           (902                  1,613   

                     Counterparty         CIK Reporting Type Report Year  
0  Morgan Stanley CapitalServices  0001070287          N-CSR          12  
1  Morgan Stanley CapitalServices  0001070287          N-CSR       

                                         Description      Date    Amount  \
0  Dow Jones CDX.EM.5 Index Receive $2,739,000\n ...  06/20/11   2739000   
1  Dow Jones CDX.EM.5 Index Receive\n  $41,080,00...  06/20/11  41080000   
2  General Motors Acceptance Corporation\n  Recei...  03/20/11   1354500   
3  General Motors Acceptance Corporation\n  Recei...  03/20/11   1354500   
4  General Motors Acceptance Corporation\n  Recei...  03/20/11   1360000   
5  Georgia Pacific Receive $531,000 upon event\n ...  12/20/10    531000   

  Appreciation(Depreciation)         CIK Reporting Type Report Year  
0                    (22,571  0001066602          N-CSR          06  
1                      16724  0001066602          N-CSR          06  
2                    (22,275  0001066602          N-CSR          06  
3                    (27,743  0001066602          N-CSR          06  
4                    (67,846  0001066602          N-CSR          06  
5                    (35,859  0001066602       

       Counterparty\n Notional amount      date         CIK Reporting Type  \
0  Deutsche Bank\n AG         1350000  09/20/15  0000944684          N-CSR   
1  Deutsche Bank\n AG         2200000  12/20/15  0000944684          N-CSR   
2  Deutsche Bank\n AG         2400000  12/20/15  0000944684          N-CSR   

  Report Year  
0          11  
1          11  
2          11  
Filename ../Data-CSR/0000944684_N-CSR_0001209286-11-000178/table1.csv
   NotionalAmount ExpirationDate                  Counterparty  \
0         800,000        6/20/16  Credit Suisse Securities LLC   
1         200,000        9/20/17  Credit Suisse Securities LLC   
2       1,300,000       12/20/14           Goldman Sachs & Co.   
3         200,000        6/20/15           Goldman Sachs & Co.   
4         100,000        3/20/16            GoldmanSachs & Co.   
5         300,000        9/20/16            GoldmanSachs & Co.   
6         200,000        6/20/16           JP MorganChase Bank   
7         100,000        

  Description Counterparty /Exchange NotionalAmounta PeriodicPaymentRate  \
0  MCDX.NA.21                   CITI         7000000                   1   
1  MCDX.NA.23                   CITI         3000000                   1   

  ExpirationDate UpfrontPremiumsPaid(Received) UnrealizedAppreciation  \
0       12/20/18                      (104,821                 173015   
1       12/20/19                         19683                   2602   

  UnrealizedDepreciation          MarketValue         CIK Reporting Type  \
0                  68194  Non InvestmentGrade  0000837274          N-CSR   
1                  22285  Non InvestmentGrade  0000837274          N-CSR   

  Report Year  
0          15  
1          15  
Filename ../Data-CSR/0000837274_N-CSR_0001193125-15-076216/table4.csv
       Counterparty Reference\n  Obligation Buy/\nSell(1)   \
0  Credit\n  Suisse                    LCDX           Sell   

  (Pay)/\n  Receive\n  Rate Expiration\n  Date Notional\n  Amount  \
0         

                       Counterparty     ReferenceEntity     ExpirationDate  \
0  Citigroup Global Markets Limited  Republic of Turkey  December 20, 2015   

  PayFixedRate (%) NotionalAmount ($) MarketValue ($)  \
0              1.0          2,700,000         (13,077   

  UnamortizedPremium(Paid)Received ($)  \
0                              (96,948   

  PeriodicPaymentsReceivable(Payable) ($) UnrealizedAppreciation($)  \
0                                  (5,400                  (115,425   

  UnrealizedDepreciation($)         CIK Reporting Type Report Year  
0                       NaN  0001352280          N-CSR          13  
Filename ../Data-CSR/0001352280_N-CSR_0001193125-13-034465/table1.csv
  Counterparty Reference Obligation Implied Fixed ExpirationDate  \
0          NaN         CDX.NA.IG-17     NaN   1.0     12/20/2016   
1          NaN         CDX.NA.IG-18     NaN   1.0     06/20/2017   
2          NaN         CDX.NA.IG-19     NaN   1.0     12/20/2017   

         Notional  

### Generate conditions for each type of Header 

In this section, we will decide the final size of the unified CSV that we are going to generate for each time user searches for something. 

In [21]:
column_headers = ["CIK", "Reporting Type", "Reporting Year", "Counterparty", "Notional Amount", "Reference Entity/Obligation", "Fixed Rate", "Expiration Date", "Appreciation/Depreciation", "Upfront Payments Paid/Received", "Implied Credit Spread", "Buy/Sell Protection", "Description"]
unified_csv = pd.DataFrame()
CIK_csv = pd.DataFrame(columns=["CIK"])
Reporting_Type_csv = pd.DataFrame(columns=["Reporting Type"])
Reporting_Year_csv = pd.DataFrame(columns=["Reporting Year"])
Counterparty_csv = pd.DataFrame(columns=["Counterparty"])
Notional_Amount_csv = pd.DataFrame(columns=["Notional Amount"])
Reference_Entity_csv = pd.DataFrame(columns=["Reference Entity/Obligation"])
Fixed_Rate_csv = pd.DataFrame(columns=["Fixed Rate"])
Expiration_Date_csv = pd.DataFrame(columns=["Expiration Date"])
Appreciation_csv = pd.DataFrame(columns=["Appreciation/Depreciation"])
Upfront_Payments_csv = pd.DataFrame(columns=["Upfront Payments Paid/Received"])
Buy_Sell_csv = pd.DataFrame(columns=["Buy/Sell Protection"])
Description_csv = pd.DataFrame(columns=["Description"]) 

#### Joining the headers and merging them to a final CSV

In [22]:
for filename in filelist:
    #Add flgas to detect changes
    CIK_Flag = 0
    ReportingType_Flag = 0
    ReportingYear_Flag = 0
    Counterparty_Flag = 0
    NotionalAmount_Flag = 0
    ReferenceEntity_Flag = 0
    FixedRate_Flag = 0
    ExpirationDate_Flag = 0
    App_Dep_Flag = 0
    Upfront_Payments_Flag = 0
    Buy_Sell_Flag = 0
    Description_Flag = 0
    
    
    
    
    #Reading and performing basic data cleaning 
    df = pd.read_csv(filename, engine='python', dtype=str)
    df = df.replace(r'\n',' ', regex=True)
    df = df.replace(r'\t',' ', regex=True)
    #Putting the list of headers in a list
    headers = list(df)
    
    #Checking for CIK in the list if not append NaN for all rows
    for header in headers:
        #CIK NUMBER
        if 'cik' in header.lower():
            CIK_Flag = 1
            for i in df[header].iteritems():
                CIK_csv = CIK_csv.append({'CIK': i[1]}, ignore_index=True)
                
    if CIK_Flag == 0:
        for i in df[header].iteritems():
            CIK_csv = CIK_csv.append({'CIK': "NaN"}, ignore_index=True)
            
    #Checking for Reporting Type in the list if not append NaN for all rows        
    for header in headers:    
        if 'reporting' in header.lower():
            ReportingType_Flag = 1
            for i in df[header].iteritems():
                Reporting_Type_csv = Reporting_Type_csv.append({'Reporting Type': i[1]}, ignore_index=True)
                
    if ReportingType_Flag == 0:
        for i in df[header].iteritems():
            Reporting_Type_csv = Reporting_Type_csv.append({'Reporting Type': "NaN"}, ignore_index=True)
            
    #Checking for Reporting Year in the list if not append NaN for all rows   
    for header in headers:
        if 'year' in header.lower():
            ReportingYear_Flag = 1
            for i in df[header].iteritems():
                Reporting_Year_csv = Reporting_Year_csv.append({'Reporting Year': i[1]}, ignore_index=True)
                
    if ReportingYear_Flag == 0:
        for i in df[header].iteritems():
            Reporting_Year_csv = Reporting_Year_csv.append({'Reporting Year': "NaN"}, ignore_index=True)
    
    #Checking for Counterparty in the list if not append NaN for all rows
    for header in headers:    
        if 'counter' in header.lower() or 'party' in header.lower() or 'obligation' in header.lower():
            Counterparty_Flag = 1
            for i in df[header].iteritems():
                Counterparty_csv = Counterparty_csv.append({'Counterparty': i[1]}, ignore_index=True)
                
    if Counterparty_Flag == 0:
        for i in df[header].iteritems():
            Counterparty_csv = Counterparty_csv.append({'Counterparty': "NaN"}, ignore_index=True)
    
    #Checking for Notional Amount in the list if not append NaN for all rows
    for header in headers:           
        if 'notional' in header.lower() or 'amount' in header.lower() or 'value' in header.lower():
            NotionalAmount_Flag = 1
            for i in df[header].iteritems():
                Notional_Amount_csv = Notional_Amount_csv.append({'Notional Amount': i[1]}, ignore_index=True)
                
    if NotionalAmount_Flag == 0:
        for i in df[header].iteritems():
            Notional_Amount_csv = Notional_Amount_csv.append({'Notional Amount': "NaN"}, ignore_index=True)
                               
    #Checking for Reference Entity/Obligation in the list if not append NaN for all rows             
    for header in headers:              
        if 'reference' in header.lower() or ('entity' in header.lower() or 'obligation' in header.lower()):
            ReferenceEntity_Flag = 1
            for i in df[header].iteritems():
                Reference_Entity_csv = Reference_Entity_csv.append({'Reference Entity/Obligation': i[1]}, ignore_index=True)
   
    if ReferenceEntity_Flag == 0:
        for i in df[header].iteritems():
            Reference_Entity_csv = Reference_Entity_csv.append({'Reference Entity/Obligation': "NaN"}, ignore_index=True)
    
    #Checking for Fixed Rate in the list if not append NaN for all rows
    for header in headers:
        if 'fixed' in header.lower() or 'rate' in header.lower():
            FixedRate_Flag = 1
            for i in df[header].iteritems():
                Fixed_Rate_csv = Fixed_Rate_csv.append({'Fixed Rate': i[1]}, ignore_index=True)
                
    if FixedRate_Flag == 0:
        for i in df[header].iteritems():
            Fixed_Rate_csv = Fixed_Rate_csv.append({'Fixed Rate': "NaN"}, ignore_index=True)  
            
     #Checking for Expiration Date in the list if not append NaN for all rows
    for header in headers:
        if 'date' in header.lower() or 'expiration' in header.lower() or 'termination' in header.lower() or 'maturity' in header.lower():
            ExpirationDate_Flag = 1
            for i in df[header].iteritems():
                Expiration_Date_csv = Expiration_Date_csv.append({'Expiration Date': i[1]}, ignore_index=True)
                
    if ExpirationDate_Flag == 0:
        for i in df[header].iteritems():
            Expiration_Date_csv = Expiration_Date_csv.append({'Expiration Date': "NaN"}, ignore_index=True)
    
    #Checking for Appreciation Depreciation in the list if not append NaN for all rows
    for header in headers:
        if 'appreciation' in header.lower() or 'depriciation' in header.lower():
            App_Dep_Flag = 1
            for i in df[header].iteritems():
                Appreciation_csv = Appreciation_csv.append({'Appreciation/Depreciation': i[1]}, ignore_index=True)
    
    if App_Dep_Flag == 0:
        for i in df[header].iteritems():
            Appreciation_csv = Appreciation_csv.append({'Appreciation/Depreciation': "NaN"}, ignore_index=True)
    
    #Checking for Upfront Payments Paid/Received in the list if not append NaN for all rows
    for header in headers:
        if 'upfront' in header.lower() or  ('payment' in header.lower() or 'premium' in header.lower()):
            Upfront_Payments_Flag = 1
            for i in df[header].iteritems():
                Upfront_Payments_csv = Upfront_Payments_csv.append({'Upfront Payments Paid/Received': i[1]}, ignore_index=True)
    
    if Upfront_Payments_Flag == 0:
        for i in df[header].iteritems():
            Upfront_Payments_csv = Upfront_Payments_csv.append({'Upfront Payments Paid/Received': "NaN"}, ignore_index=True)
    
    
    #Checking for Buy/Sell Protection in the list if not append NaN for all rows
    for header in headers:
        if 'buy' in header.lower() or  'sell' in header.lower() or 'protection' in header.lower():
            Buy_Sell_Flag = 0
            for i in df[header].iteritems():
                Buy_Sell_csv = Buy_Sell_csv.append({'Buy/Sell Protection': i[1]}, ignore_index=True)
    if Buy_Sell_Flag == 0:
        for i in df[header].iteritems():
            Buy_Sell_csv = Buy_Sell_csv.append({'Buy/Sell Protection': "NaN"}, ignore_index=True)
    
    #Checking for Description in the list if not append NaN for all rows
    for header in headers:
        if 'description' in header.lower():
            Description_Flag = 1
            for i in df[header].iteritems():
                Description_csv = Description_csv.append({'Description': i[1]}, ignore_index=True)
    if Description_Flag == 0:
        for i in df[header].iteritems():
            Description_csv = Description_csv.append({'Description': "NaN"}, ignore_index=True)         

Let's check the final result

In [11]:
Buy_Sell_csv

,Buy/Sell Protection
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,Buy
6,NaN
7,Buy
8,Buy
9,NaN


In [25]:
unified_csv = CIK_csv.join(Reporting_Type_csv)
unified_csv = unified_csv.join(Reporting_Year_csv)
unified_csv = unified_csv.join(Counterparty_csv)
unified_csv = unified_csv.join(Notional_Amount_csv)
unified_csv = unified_csv.join(Reference_Entity_csv)
unified_csv = unified_csv.join(Expiration_Date_csv)
unified_csv = unified_csv.join(Appreciation_csv)
unified_csv = unified_csv.join(Upfront_Payments_csv)
unified_csv = unified_csv.join(Buy_Sell_csv)
unified_csv = unified_csv.join(Description_csv)

# Setting column type to str
CIK_csv = CIK_csv.astype(str)
Reporting_Type_csv = Reporting_Type_csv.astype(str)

In [26]:
unified_csv.to_csv("final_csv.csv", index=False)

In [27]:
unified_csv

,CIK,Reporting Type,Reporting Year,Counterparty,Notional Amount,Reference Entity/Obligation,Expiration Date,Appreciation/Depreciation,Upfront Payments Paid/Received,Buy/Sell Protection,Description
0,0000315774,N-CSR,17,Barclays,"10,000,000","Eastman Chemical Co, 7.60%, 02/01/27",12/20/21,NaN,"35,394",NaN,NaN
1,0000315774,N-CSR,17,Barclays,"5,000,000","Host Hotels & Resorts, 4.75%, 03/01/23",12/20/21,NaN,"50,649",NaN,NaN
2,0000315774,N-CSR,17,Barclays,"10,000,000","Macy's Retail Holdings, 7.45%, 07/15/17",12/20/21,NaN,"507,117",NaN,NaN
3,0000315774,N-CSR,17,BNP Paribas,"10,000,000","Host Hotels & Resorts, 4.75%, 03/01/23",06/20/22,NaN,"(24,159",NaN,NaN
4,0000315774,N-CSR,17,BNP Paribas,"5,000,000","International Paper Co, 7.50%, 08/15/21",12/20/21,NaN,"(49,431",NaN,NaN
5,0000315774,N-CSR,17,BNP Paribas,"5,000,000","Marriott International, Inc., 3.00%, 03/01/19",12/20/21,NaN,"(88,168",NaN,NaN
6,0000315774,N-CSR,17,BNP Paribas,"10,000,000","Ryder Systems Inc, 2.55%, 06/01/19",12/20/21,NaN,"(22,277",NaN,NaN
7,0000315774,N-CSR,17,Goldman Sachs,"5,000,000","Host Hotels & Resorts, 4.75%, 03/01/23",12/20/21,NaN,"28,750",NaN,NaN
8,0000315774,N-CSR,17,Goldman Sachs,"5,000,000","Nordstrom Inc, 6.95%, 03/15/28",06/20/22,NaN,"156,817",NaN,NaN
9,0000315774,N-CSR,17,Goldman Sachs,"10,000,000","Ryder Systems Inc, 2.55%, 06/01/19",06/20/22,NaN,"(92,399",NaN,NaN
